Variables are manipulated via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Unlike tf.Tensor objects, a tf.Variable exists outside the context of a single session.run call.

The best way to create a variable is to call the tf.get_variable function. This function requires you to specify the Variable's name.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
sess = tf.Session()

In [2]:
# This creates a variable named "my_variable" which is a three-dimensional tensor with shape [1, 2, 3]. 
# This variable will, by default, have the dtype tf.float32 and its initial value will be randomized via 
# tf.glorot_uniform_initializer.
my_variable = tf.get_variable("my_variable", [1, 2, 3])

# You may optionally specify the dtype and initializer to tf.get_variable. For example:
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32,
  initializer=tf.zeros_initializer)

# TensorFlow provides many convenient initializers. 
# Alternatively, you may initialize a tf.Variable to have the value of a tf.Tensor. For example:
other_variable = tf.get_variable("other_variable", dtype=tf.int32,
  initializer=tf.constant([23, 42]))

Collections: Are named lists of tensors or other objects, such as tf.Variable instances.

By default every tf.Variable gets placed in the following two collections:

    1) tf.GraphKeys.GLOBAL_VARIABLES --- variables that can be shared across multiple devices,
    2) tf.GraphKeys.TRAINABLE_VARIABLES --- variables for which TensorFlow will calculate gradients.

In [3]:
# If you don't want a variable to be trainable, add it to the tf.GraphKeys.LOCAL_VARIABLES collection instead. 
# For example, the following snippet demonstrates how to add a variable named my_local to this collection:
my_local = tf.get_variable("my_local", shape=(),
collections=[tf.GraphKeys.LOCAL_VARIABLES])

# or

my_non_trainable = tf.get_variable("my_non_trainable",
                                   shape=(),
                                   trainable=False)

tf.add_to_collection("my_collection_name", my_local) # add  my_local to own collection (implicitly initalized)
tf.get_collection("my_collection_name") # retrieve from collection (returns variable of tensors)

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

Before you can use a variable, it must be initialized. If you are programming in the low-level TensorFlow API (that is, you are explicitly creating your own graphs and sessions), you must explicitly initialize the variables.

In [4]:
sess.run(tf.global_variables_initializer()) # Now all variables 'should' be initialized.

#You can also ask which variables have still not been initialized. 
# For example, the following code prints the names of all variables which have not yet been initialized:
print(sess.run(tf.report_uninitialized_variables()))

sess.run(my_local.initializer)
print(sess.run(tf.report_uninitialized_variables()))    #Since local was not being initalized automaticall

[b'my_local']
[]


In [5]:
print(tf.get_collection("my_collection_name"))
print(tf.get_collection("my_collection_name")[0].eval(sess)) # printing out my_local variable from collection

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]
1.1563798


In [6]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
#w = tf.get_variable("w", initializer=v + 1) #throws an error because v is not initalized here
w = tf.get_variable("w", initializer=v.initialized_value() + 1)
print(sess.run(tf.report_uninitialized_variables()))

[b'v' b'w']


In [7]:
sess.run(tf.global_variables_initializer())
print(sess.run(tf.report_uninitialized_variables())) #now v & w are initalized

[]


In [8]:
z = tf.get_variable("z", initializer=v+w)
#now z works since v & w are initalized, but this is bad practice.

To use the value of a tf.Variable in a TensorFlow graph, simply treat it like a normal tf.Tensor

To assign a value to a variable, use the methods assign, assign_add, and friends in the tf.Variable class.

Because variables are mutable it's sometimes useful to know what version of a variable's value is being used at any point in time. To force a re-read of the value of a variable after something has happened, you can use tf.Variable.read_value.

In [9]:
f = tf.get_variable("f", shape=(), initializer=tf.zeros_initializer())
k = f + 1  # w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value.
        
g = tf.get_variable("g", shape=(), initializer=tf.zeros_initializer())
assignment = g.assign_add(1)
sess.run(tf.global_variables_initializer())
sess.run(assignment)  # or assignment.op.run(), or assignment.eval()

h = tf.get_variable("h", shape=(), initializer=tf.zeros_initializer())
assignment = h.assign_add(1)
with tf.control_dependencies([assignment]):
  w = h.read_value()  # w is guaranteed to reflect v's value after the
                      # assign_add operation.

In [11]:
print(g.eval(sess))
# print(g.eval(sess)) #would throw an error since g was not initalized

1.0
